# Image classifier
This notebook has been created to practice lesson two of Practical Deep Learning for Coders available on https://course.fast.ai/. Thanks to Jeremy Howard for this amazing course!

## What are we going to make?
At the end of this notebook we are going to have a model that can classify kitchen appliances such as washing machine, dishwasher, gas range.

First, let's import what we need.

In [9]:
from fastai.vision import *

## Preparing data
### Making directories
First, let's make appropriate directories to download and save images of each calss.

In [10]:
classes = ['washing-machine','dishwasher', 'gas-range']
path = Path('data/homeAppliances')
for myClass in classes:
  dest = path/myClass
  dest.mkdir(parents=True, exist_ok=True)

Let's see it worked!

In [11]:
path.ls()

[PosixPath('data/homeAppliances/washing-machine'),
 PosixPath('data/homeAppliances/dishwasher'),
 PosixPath('data/homeAppliances/gas-range')]

Now, you can search on google to find images to train your model. Search in google images and then open console and write this javaScript code to save csv file of link to images google found for you!

```javascript
urls=Array.from(document.querySelectorAll('.rg_i')).map(el=> el.hasAttribute('data-src')?el.getAttribute('data-src'):el.getAttribute('data-iurl'));
window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));
```

Then upload these csv file on to your working directory to be able to download images in next step.

## Downloading Images

Now, we download and save images.

In [37]:
download_images('./washing-machine.csv', 'data/homeAppliances/washing-machine', max_pics=200)
download_images('./dishwasher.csv', 'data/homeAppliances/dishwasher', max_pics=200)
download_images('./gas-range.csv', 'data/homeAppliances/gas-range', max_pics=200)

In [38]:
# if you have problem downloading files try this to see exception
# download_images((csv_file_path/myClass), dest, max_pics=20, max_workers=0)

Let's remove images that cannot be opened.

In [49]:
classes = ['washing-machine','dishwasher', 'gas-range']
path = Path('data/homeAppliances')
for myClass in classes:
  dest = path/myClass
  print(myClass)
  verify_images(dest, delete=True, max_size=500)

washing-machine


dishwasher


gas-range
